- Load different HSI L1 data
- Retrieve methane enhancement
- Folium map preview

For the batch-processing, please check the **Batch Processing** section of Documentation.

In [1]:
import os
import sys
import folium

from glob import glob
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from hypergas import Hyper
from hypergas.folium_map import Map

In [2]:
data_dir = '../test_data/Norte_III/'

## EnMAP

In [3]:
filename = glob(data_dir+'ENMAP01-____L1B-DT0000052417_20231122T14*.ZIP')
# get absolute path in case the notebook searches through package path
filename  = os.path.abspath(filename[0])

hyp_enmap = Hyper([filename], reader='hsi_l1b') # "hsi_l1b", "emit_l1b", "hyc_l1"

hyp_enmap.load()

/home/xinz/Documents/githab/satpy_hyper/satpy/readers/hsi_l1b.py:53: RuntimeWarning: No digital elevation model provided. Note that this may cause uncertainties, e.g., in the atmospheric correction and the orthorectification.
  self.cfg = EnPTConfig(**config_minimal)


In [4]:
hyp_enmap.retrieve(wvl_intervals=[1300, 2500])
ch4_swir = hyp_enmap.scene['ch4']
hyp_enmap.retrieve(wvl_intervals=[2100, 2450])
ch4 = hyp_enmap.scene['ch4']

# calculate ch4_comb
diff = ch4_swir - ch4
scale = ch4.std()/ch4_swir.std()
# scale if ch4_swir < ch4
hyp_enmap.scene['ch4_comb'] = ch4.where(diff > 0, ch4_swir*scale).rename('ch4_comb')

In [5]:
hyp_enmap.scene['ch4_denoise'] = hyp_enmap.denoise(varname='ch4')
hyp_enmap.scene['ch4_comb_denoise'] = hyp_enmap.denoise(varname='ch4_comb')

In [6]:
def ortho_enmap(hyp):
    rgb_corr = hyp.terrain_corr(varname='rgb', rpcs=hyp.scene['rpc_coef_vnir'].sel(bands_vnir=650, method='nearest').item())
    u10 = hyp.terrain_corr(varname='u10', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())
    v10 = hyp.terrain_corr(varname='v10', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())
    
    ch4_corr = hyp.terrain_corr(varname='ch4', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())
    ch4_comb_corr = hyp.terrain_corr(varname='ch4_comb', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())
    ch4_denoise_corr = hyp.terrain_corr(varname='ch4_denoise', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())
    ch4_comb_denoise_corr = hyp.terrain_corr(varname='ch4_comb_denoise', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())

    rgb = rgb_corr.interp_like(ch4_corr)
    rgb.attrs['area'] = ch4_corr.attrs['area']

    ch4 = ch4_corr.isel(bands=0)
    ch4_comb = ch4_comb_corr.isel(bands=0)
    ch4_denoise = ch4_denoise_corr.isel(bands=0)
    ch4_comb_denoise = ch4_comb_denoise_corr.isel(bands=0)

    hyp.scene['u10'] = u10
    hyp.scene['v10'] = v10
    hyp.scene['rgb'] = rgb
    hyp.scene['ch4'] = ch4
    hyp.scene['ch4_comb'] = ch4_comb
    hyp.scene['ch4_denoise'] = ch4_denoise
    hyp.scene['ch4_comb_denoise'] = ch4_comb_denoise

In [7]:
ortho_enmap(hyp_enmap)

In [8]:
vnames = ['u10', 'v10', 'rgb', 'ch4', 'ch4_comb', 'ch4_denoise', 'ch4_comb_denoise']

# export to Dataset
ds_enmap = hyp_enmap.scene.to_xarray(datasets=vnames)
ds_enmap.attrs['filename'] = filename

# export to netcdf file
# hyp_enmap.scene.save_datasets(datasets=vnames, filename='test.nc', writer='cf')

In [9]:
ds_enmap

<xarray.Dataset>
Dimensions:           (y: 1182, x: 1302, bands: 3, source: 2)
Coordinates:
  * y                 (y) float64 6.196e+06 6.196e+06 ... 6.16e+06 6.16e+06
  * x                 (x) float64 3.322e+05 3.322e+05 ... 3.712e+05 3.712e+05
    longitude         (y, x) float64 -58.83 -58.82 -58.82 ... -58.41 -58.41
    latitude          (y, x) float64 -34.37 -34.37 -34.37 ... -34.69 -34.69
  * source            (source) object 'ERA5' 'GEOS-FP'
Dimensions without coordinates: bands
Data variables:
    hsi_utm           int64 0
    ch4               (y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    ch4_comb          (y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    ch4_comb_denoise  (y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    ch4_denoise       (y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    rgb               (bands, y, x) float64 nan nan nan nan ... nan nan nan nan
    u10               (source, y, x) float64 nan nan nan nan ... nan nan nan nan
    v10               (source, y, x) float64 nan nan nan nan ... nan nan nan nan
Attributes:
    history:      Created by pytroll/satpy on 2024-03-15 20:30:57.283893
    Conventions:  CF-1.7
    filename:     /home/xinz/Documents/githab/HyperCH4/test_data/Norte_III/EN...

In [ ]:
m_enmap = Map(ds_enmap, ['rgb', 'ch4', 'ch4_comb','ch4_comb_denoise'])
m_enmap.initialize()
m_enmap.plot(show_layers=[False, False, False, True],
       opacities=[0.9, 0.7, 0.7, 0.7])

# show the map in notebook
layer_control = folium.LayerControl(collapsed=False, position='topleft', draggable=True)
m_enmap.map.add_child(layer_control)

# export to html file
# m_enmap.export()

## EMIT

In [11]:
filename_emit = glob(data_dir+'EMIT_L1B_*20231124*.nc')
# get absolute path in case the notebook searches through package path
filename_emit  = [os.path.abspath(filename) for filename in filename_emit]

hyp_emit = Hyper(filename_emit, reader='emit_l1b') # "hsi_l1b", "emit_l1b", "hyc_l1"

hyp_emit.load()

/home/xinz/miniconda3/envs/hyper/lib/python3.11/site-packages/xarray/core/indexing.py:1443: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/xinz/miniconda3/envs/hyper/lib/python3.11/site-packages/dask/array/core.py:1712: FutureWarning: The `numpy.trapz` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/xinz/miniconda3/envs/hyper/lib/python3.11/site-packages/dask/array/core.py:1712: FutureWarning: The `numpy.trapz` function is not implemented by Dask array. You may want to u

In [12]:
hyp_emit.retrieve(wvl_intervals=[1300, 2500])
ch4_swir = hyp_emit.scene['ch4']
hyp_emit.retrieve(wvl_intervals=[2100, 2450])
ch4 = hyp_emit.scene['ch4']

# calculate ch4_comb
diff = ch4_swir - ch4
scale = ch4.std()/ch4_swir.std()
# scale if ch4_swir < ch4
hyp_emit.scene['ch4_comb'] = ch4.where(diff > 0, ch4_swir*scale).rename('ch4_comb')

/home/xinz/miniconda3/envs/hyper/lib/python3.11/site-packages/xarray/core/indexing.py:1443: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [13]:
hyp_emit.scene['ch4_denoise'] = hyp_emit.denoise(varname='ch4')
hyp_emit.scene['ch4_comb_denoise'] = hyp_emit.denoise(varname='ch4_comb')

In [14]:
def ortho_emit(hyp):
    # orthorectification
    u10_corr = hyp.terrain_corr(varname='u10')
    v10_corr = hyp.terrain_corr(varname='v10')
    rgb_corr = hyp.terrain_corr(varname='rgb')
    ch4_corr = hyp.terrain_corr(varname='ch4')
    ch4_comb_corr = hyp.terrain_corr(varname='ch4_comb')
    ch4_denoise_corr = hyp.terrain_corr(varname='ch4_denoise')
    ch4_comb_denoise_corr = hyp.terrain_corr(varname='ch4_comb_denoise')

    # save into Scene
    hyp.scene['u10'] = u10_corr
    hyp.scene['v10'] = v10_corr
    hyp.scene['rgb'] = rgb_corr
    hyp.scene['ch4'] = ch4_corr.isel(bands=0)
    hyp.scene['ch4_comb'] = ch4_comb_corr.isel(bands=0)
    hyp.scene['ch4_denoise'] = ch4_denoise_corr.isel(bands=0)
    hyp.scene['ch4_comb_denoise'] = ch4_comb_denoise_corr.isel(bands=0)

In [15]:
ortho_emit(hyp_emit)

In [16]:
vnames = ['u10', 'v10', 'rgb', 'ch4', 'ch4_comb', 'ch4_denoise', 'ch4_comb_denoise']

# export to Dataset
ds_emit = hyp_emit.scene.to_xarray(datasets=vnames)

ds_emit.attrs['filename'] = filename_emit[0]

# export to netcdf file
# hyp_emit.scene.save_datasets(datasets=vnames, filename='test.nc', writer='cf')

In [17]:
ds_emit

<xarray.Dataset>
Dimensions:           (y: 1954, x: 2010, bands: 3, source: 2)
Coordinates:
  * y                 (y) float64 6.219e+06 6.219e+06 ... 6.102e+06 6.102e+06
  * x                 (x) float64 2.54e+05 2.541e+05 ... 3.745e+05 3.746e+05
    longitude         (y, x) float64 -59.67 -59.67 -59.67 ... -58.38 -58.38
    latitude          (y, x) float64 -34.14 -34.14 -34.14 ... -35.22 -35.22
  * source            (source) object 'ERA5' 'GEOS-FP'
Dimensions without coordinates: bands
Data variables:
    EMIT_utm          int64 0
    ch4               (y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    ch4_comb          (y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    ch4_comb_denoise  (y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    ch4_denoise       (y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    rgb               (bands, y, x) float64 nan nan nan nan ... nan nan nan nan
    u10               (source, y, x) float64 nan nan nan nan ... nan nan nan nan
    v10               (source, y, x) float64 nan nan nan nan ... nan nan nan nan
Attributes:
    history:      Created by pytroll/satpy on 2024-03-15 20:32:59.717307
    Conventions:  CF-1.7
    filename:     /home/xinz/Documents/githab/HyperCH4/test_data/Norte_III/EM...

In [ ]:
m_emit = Map(ds_emit, ['rgb', 'ch4', 'ch4_comb','ch4_comb_denoise'])
m_emit.initialize()
m_emit.plot(show_layers=[False, False, False, True],
       opacities=[0.9, 0.7, 0.7, 0.7])

# show the map in notebook
layer_control = folium.LayerControl(collapsed=False, position='topleft', draggable=True)
m_emit.map.add_child(layer_control)

# export to html file
# m_emit.export()

## PRISMA

In [19]:
filename_prisma = glob(data_dir+'PRS_L1_STD_OFFL_202308311400*.zip')[0]
# get absolute path in case the notebook searches through package path
filename_prisma = os.path.abspath(filename_prisma)

hyp_prisma = Hyper([filename_prisma], reader='hyc_l1') # "hsi_l1b", "emit_l1b", "hyc_l1"

hyp_prisma.load()

/home/xinz/miniconda3/envs/hyper/lib/python3.11/site-packages/xarray/core/indexing.py:1443: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/xinz/miniconda3/envs/hyper/lib/python3.11/site-packages/dask/array/core.py:1712: FutureWarning: The `numpy.trapz` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/xinz/miniconda3/envs/hyper/lib/python3.11/site-packages/dask/array/core.py:1712: FutureWarning: The `numpy.trapz` function is not implemented by Dask array. You may want to u

In [20]:
hyp_prisma.retrieve(wvl_intervals=[1300, 2500])
ch4_swir = hyp_prisma.scene['ch4']
hyp_prisma.retrieve(wvl_intervals=[2100, 2450])
ch4 = hyp_prisma.scene['ch4']

# calculate ch4_comb
diff = ch4_swir - ch4
scale = ch4.std()/ch4_swir.std()
# scale if ch4_swir < ch4
hyp_prisma.scene['ch4_comb'] = ch4.where(diff > 0, ch4_swir*scale).rename('ch4_comb')

In [21]:
# remove the bands dim for ch4
hyp_prisma.scene['ch4'] = hyp_prisma.scene['ch4'].isel(bands=0)
hyp_prisma.scene['ch4_comb'] = hyp_prisma.scene['ch4_comb'].isel(bands=0)

In [22]:
hyp_prisma.scene['ch4_denoise'] = hyp_prisma.denoise(varname='ch4')
hyp_prisma.scene['ch4_comb_denoise'] = hyp_prisma.denoise(varname='ch4_comb')

In [23]:
vnames = ['u10', 'v10', 'rgb', 'ch4', 'ch4_comb', 'ch4_denoise', 'ch4_comb_denoise']

# export to Dataset
ds_prisma = hyp_prisma.scene.to_xarray(datasets=vnames)

ds_prisma.attrs['filename'] = filename_prisma

# export to netcdf file
# hyp_prisma.scene.save_datasets(datasets=vnames, filename='test.nc', writer='cf')

In [24]:
ds_prisma

<xarray.Dataset>
Dimensions:           (y: 1000, x: 1000, bands: 3, source: 2)
Coordinates:
    longitude         (y, x) float32 -58.48 -58.48 -58.48 ... -58.88 -58.88
    latitude          (y, x) float32 -34.48 -34.48 -34.48 ... -34.68 -34.68
  * bands             (bands) int64 650 560 470
  * source            (source) object 'ERA5' 'GEOS-FP'
Dimensions without coordinates: y, x
Data variables:
    ch4               (y, x) float32 -2.729 155.9 -521.5 ... 110.8 -88.55 125.9
    ch4_comb          (y, x) float32 -1.731e+03 -711.7 -521.5 ... -88.55 -174.8
    ch4_comb_denoise  (y, x) float32 -1.663e+03 -677.6 -552.8 ... -62.72 -93.99
    ch4_denoise       (y, x) float32 21.35 50.19 -426.7 ... 111.0 8.819 28.73
    rgb               (bands, y, x) float64 0.0 0.0 0.0 ... 0.6416 0.6345 0.6343
    u10               (source, y, x) float64 1.769 1.769 ... -0.01068 -0.01115
    v10               (source, y, x) float64 1.747 1.746 1.746 ... 2.035 2.035
Attributes:
    history:      Created by pytroll/satpy on 2024-03-15 20:33:53.564122
    Conventions:  CF-1.7
    filename:     /home/xinz/Documents/githab/HyperCH4/test_data/Norte_III/PR...

In [ ]:
m_prisma = Map(ds_prisma, ['rgb', 'ch4', 'ch4_comb', 'ch4_comb_denoise'])
m_prisma.initialize()
m_prisma.plot(show_layers=[False, False, False, True],
       opacities=[0.9, 0.7, 0.7, 0.7])

# show the map in notebook
layer_control = folium.LayerControl(collapsed=False, position='topleft', draggable=True)
m_prisma.map.add_child(layer_control)

# export to html file
# m_prisma.export()